# 03 — Data Cleaning
**ClinicalTrials.gov Dataset**  
*Author: John Seaton*  
*Last updated: 2025-12-14*

---


## 1. Scope & Role of This Notebook

**Project context:** This notebook performs a complete, systematic cleaning of the ClinicalTrials.gov dataset to prepare it for downstream analysis of global drug development patterns.

**Data source:** ClinicalTrials.gov (public registry of clinical studies)  
**Scope / time range:** This project uses the full available ClinicalTrials.gov dataset, covering the entire historical range of registered trials   
**Primary key:** `NCT Number` (unique trial identifier used to link derived tables)

**Inputs:**  
- `data/raw/ClinicalTrials/ctg-studies.csv`

**Outputs (written in Section 14):**  
- `data/processed/clean_trials.csv` (trial-level cleaned dataset)  
- `data/processed/clean_interventions.csv` (long-format interventions)  
- `data/processed/clean_conditions.csv` (long-format conditions)  
- `data/processed/clean_locations.csv` (long-format locations + extracted country)

**Key filters / constraints:**  
- The primary dataset is filtered to retain only trials containing **DRUG** and/or **BIOLOGICAL** interventions to focus analysis on drug development activity.

**Goals:**   
- Standardize critical fields such as trial phases, locations, date columns, interventions, conditions, and enrollment values
- Resolve inconsistences arising from formatting differences, categorical variants and and grouped values
- Handle missing, irregular, or redundant data using transparent and reproducible transformations
- Engineer derived columns that improve analytical clarity and support futher exploratory or modeling efforts
- Generate a fully cleaned, versioned dataset (CSV) for use in subsequent notebooks and pipeline stages

All transformations in this notebook are documented, reproducible, and reversible.
While new datasets are created through this notebook, the original raw dataset remains intact.

## 2. Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')

## 3. Load Raw Data

In [2]:
df = pd.read_csv('../data/raw/ClinicalTrials/ctg-studies.csv')
df.shape

(530028, 22)

## 4. High Level DataFrame Exploration

In [3]:
# Show data types
df.dtypes

NCT Number                     object
Study Title                    object
Study URL                      object
Study Status                   object
Conditions                     object
Interventions                  object
Primary Outcome Measures       object
Secondary Outcome Measures     object
Sponsor                        object
Collaborators                  object
Age                            object
Phases                         object
Enrollment                    float64
Funder Type                    object
Study Type                     object
Study Design                   object
Start Date                     object
Primary Completion Date        object
Completion Date                object
First Posted                   object
Locations                      object
Study Documents                object
dtype: object

Dates will need to be converted from object to datetime later on.

In [4]:
# Show column names
df.columns.tolist()


['NCT Number',
 'Study Title',
 'Study URL',
 'Study Status',
 'Conditions',
 'Interventions',
 'Primary Outcome Measures',
 'Secondary Outcome Measures',
 'Sponsor',
 'Collaborators',
 'Age',
 'Phases',
 'Enrollment',
 'Funder Type',
 'Study Type',
 'Study Design',
 'Start Date',
 'Primary Completion Date',
 'Completion Date',
 'First Posted',
 'Locations',
 'Study Documents']

In [5]:
# High-level missingness snapshot
df.isna().mean().sort_values(ascending=False).head(15)

Study Documents               0.909578
Collaborators                 0.675219
Phases                        0.614666
Secondary Outcome Measures    0.269754
Interventions                 0.098470
Locations                     0.081267
Primary Completion Date       0.032085
Primary Outcome Measures      0.025414
Completion Date               0.023299
Enrollment                    0.007309
Conditions                    0.000006
Study Status                  0.000000
NCT Number                    0.000000
Study Title                   0.000000
Study URL                     0.000000
dtype: float64

The columns containing missing data will be addressed in the following ways:

`Study Documents` (90.9% missing): Drop column. Not needed for analysis.

`Collaborators` (67.5% missing): Drop column. Duplicative with Sponsor/Funder Type.

`Phases` (61.5% missing): Keep column, leave missing values as-is. Missingness expected for observational studies.

`Primary Outcome Measures` (23.3% missing) and `Secondary Outcome Measures` (27.0% missing): Drop these columns. Helpful but not essential data.

`Interventions` (9.8% missing): Keep column. Investigate missing cases during cleaning. 

`Locations` (8.1% missing): Keep column. Investigate missing cases during cleaning. 

`Primary Completion Date` (3.2% missing) and `Completion Date` (2.3% missing): Keep one of these. Missing data likely from ongoing trials. 

`Enrollment` (0.7% missing): Keep column. Investigate missing cases during cleaning. 

There is a singular trial with missing Conditions data. This will be removed.

In [6]:
# Small sample of the starting dataframe
df.sample(3)

,NCT Number,Study Title,Study URL,Study Status,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,Sponsor,Collaborators,...,Enrollment,Funder Type,Study Type,Study Design,Start Date,Primary Completion Date,Completion Date,First Posted,Locations,Study Documents
285416,NCT04730557,Reducing Obesity and Cartilage Compression in ...,https://clinicaltrials.gov/study/NCT04730557,COMPLETED,Obesity|Osteoarthritis|Knee Osteoarthritis,BEHAVIORAL: Weight Loss,Change from baseline in cartilage strain/thick...,Change from baseline in cartilage composition ...,Duke University,NaN,...,88.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2020-06-05,2023-10-26,2024-05-17,2021-01-29,"Duke University Medical Center, Durham, North ...",NaN
434463,NCT05603624,Effect of Sterile Versus Clean Gloves Intrapar...,https://clinicaltrials.gov/study/NCT05603624,TERMINATED,Chorioamnionitis|Intrauterine Infection|Postpa...,PROCEDURE: Cervical examination to assess labor,Chorioamnionitis or intraamniotic or intrauter...,"Postpartum endometritis, As defined by ACOG co...",Eastern Virginia Medical School,NaN,...,163.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2021-09-02,2022-09-01,2022-09-01,2022-11-02,"Eastern Virginia Medical School, Norfolk, Virg...",NaN
191593,NCT04878627,Role of CBD in Regulating Meal Time Anxiety in...,https://clinicaltrials.gov/study/NCT04878627,ACTIVE_NOT_RECRUITING,Anorexia Nervosa,DRUG: Cannabidiol|DRUG: Placebo,Committee of Clinical Investigations UKU-Side ...,NaN,"University of California, San Diego",NaN,...,40.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2022-01-20,2026-06,2026-06,2021-05-07,"University of California San Diego, San Diego,...",NaN


## 5. Cleaning Overview

This notebooks applies a series of cleaning steps to standardize the ClinicalTrials.gov dataframe for analysis. Key transformations include (in order): 

- converting all date fields to proper datetime format
- dropping fields that are redundant or irrelevant for the analysis of drug development pipelines
- normalizing categorical fields such as trial phases and funder types
- parsing and classifying interventions
- engineering a derived field for combined drug and biological interventions
- cleaning and structuring the conditions and locations fields
- verifying enrollment values
- then exporting the final processed dataframes to CSVs

Each transformation or cleaning step is applied sequentially below and documented for reproducability.

## 6. Standardize Date Columns

In [7]:
# Convert all four data columns from object type to datetime format

date_cols = ['Start Date', 'Primary Completion Date', 'Completion Date', 'First Posted']

# Store raw string backups
for col in date_cols:
    df[f'{col} (raw)'] = df[col]

# Convert to datetime
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

 # Add '15' (monthly-midpoint) to dates with missing day values to convert YYYY-MM to YYYY-MM-15
 # This standardizes date format for later analysis
for col in date_cols:
    raw_col = f'{col} (raw)'
    
    # Locate dates with YYYY-MM formatting
    month_only = df[raw_col].astype(str).str.match(r'^\d{4}-\d{2}$')
    
    df.loc[month_only, col] = pd.to_datetime(df.loc[month_only, raw_col] + '-15',errors='coerce')

In [8]:
# Confirm dates have been coverted to datetime
print(df[date_cols].dtypes)
print()

Start Date                 datetime64[ns]
Primary Completion Date    datetime64[ns]
Completion Date            datetime64[ns]
First Posted               datetime64[ns]
dtype: object



In [9]:
# Check on missing date values after above transformations
df[date_cols].isna().mean() * 100

Start Date                 0.000000
Primary Completion Date    3.208510
Completion Date            2.329877
First Posted               0.000000
dtype: float64

In [10]:
# Convert % missing into absolute counts
for col in ['Primary Completion Date', 'Completion Date']:
    missing_count = df[col].isna().sum()
    total = len(df)
    print(f"{col}: {missing_count} missing out of {total} "
          f"({missing_count / total * 100:.2f}%)")

Primary Completion Date: 17006 missing out of 530028 (3.21%)
Completion Date: 12349 missing out of 530028 (2.33%)


In [11]:
# Show the most common values in the Completion Date fields
for col in ['Primary Completion Date', 'Completion Date']:
    print(df[col].value_counts(dropna=False).head(3))

Primary Completion Date
NaT           17006
2025-12-31     3768
2025-12-15     3116
Name: count, dtype: int64
Completion Date
NaT           12349
2025-12-31     4188
2025-12-15     3307
Name: count, dtype: int64


This validates that all remaining missing values for the Completion Date columns were missing in the original clinicaltrials.gov dataset, and were not created during the cleaning or conversion steps above. The missing values are relatively small (2-3%) and most likely represent the trials that are ongoing at the time the dataset was downloaded. The four date columns have been converted to datetime format, the YYYY-MM data values have been converted to YYYY-MM-15 format, and all remaining missing values in these columns are NaT (not a Time) and will be left as-is.

## 7. Dropping Data

In [12]:
# Check existing dataframe shape before dropping columns
df.shape


(530028, 26)

In [13]:
df = df.drop('Study Title', axis=1)
df = df.drop('Study Status', axis=1)
df = df.drop('Study URL', axis=1)
df = df.drop('Study Documents', axis=1)
df = df.drop('Primary Outcome Measures', axis=1)
df = df.drop('Secondary Outcome Measures', axis=1)
df = df.drop('Collaborators', axis=1)
df = df.drop('Age', axis=1)
df = df.drop('Completion Date', axis=1)
df = df.drop('First Posted', axis=1)
df = df.drop('Study Design', axis=1)
# Drop raw date columns created earlier in section 6
df = df.drop('First Posted (raw)', axis=1)
df = df.drop('Start Date (raw)', axis=1)
df = df.drop('Primary Completion Date (raw)', axis=1)
df = df.drop('Completion Date (raw)', axis=1)

In [14]:
# Check existing dataframe shape after dropping columns
df.shape

(530028, 11)

In [15]:
# Show what columns remain
df.columns.tolist()

['NCT Number',
 'Conditions',
 'Interventions',
 'Sponsor',
 'Phases',
 'Enrollment',
 'Funder Type',
 'Study Type',
 'Start Date',
 'Primary Completion Date',
 'Locations']

Eleven of the original fields were succesfully dropped from dataframe `df` due to their irrelevance to the primary analysis. There are now eleven columns remaining, and one more will be added in section 9.

## 8. Clean and Normalize Categorical Columns

This section exists to identify categorical columns that require normalization and light cleaning (inconsistent casing, delimiter usage, or fields with multiple values). The goal is to ensure values are internally consistent within each field to ensure values are suitable for later analysis. Only light work is performed here in section 8, complex transformations are taken on later in sections dedicated to each field. 

In [16]:
#  Show most unique values for each category
df.select_dtypes(include='object').nunique().sort_values()

Study Type            2
Phases                7
Funder Type           9
Sponsor           46212
Conditions       214914
Locations        289919
Interventions    396586
NCT Number       530028
dtype: int64

Low cardinality categories (categories containing small number of unique elements in the set) that will be addressed here include Study Type, Age, Phases, and Funder Type. Remaining categories have a huge variety of unique elements, with values in the thousands. Let's investigate the unique elements in each of the four categories.

In [17]:
df['Study Type'].value_counts(dropna=False)


Study Type
INTERVENTIONAL    406244
OBSERVATIONAL     123784
Name: count, dtype: int64

No work to be done here, two unique values. Leaving as-is.

In [18]:
df['Phases'].value_counts(dropna=False)


Phases
NaN              325790
PHASE2            59379
PHASE1            44736
PHASE3            39119
PHASE4            33139
PHASE1|PHASE2     15396
PHASE2|PHASE3      6972
EARLY_PHASE1       5497
Name: count, dtype: int64

The Phases column has some cases of combined values, such as Phase 1|Phase 2, which will causes issues with analysis. These combined values are scenarios where the trial phases are running concurrently. For the sake of this analysis, the trial phase will be conerted to the higher of the two values, i.e. Phase 1|Phase 2 --> Phase 2.

In [19]:
# Replace the inconsistent values 
corrected_values = {'EARLY_PHASE1': 'PHASE1', 'PHASE1|PHASE2': 'PHASE2', 'PHASE2|PHASE3': 'PHASE3'}
df['Phases'] = df['Phases'].replace(corrected_values)

In [20]:
# Show corrected Phases values
df['Phases'].value_counts(dropna=False).head()

Phases
NaN       325790
PHASE2     74775
PHASE1     50233
PHASE3     46091
PHASE4     33139
Name: count, dtype: int64

The Phases column is now ready for further analysis. All combined and inconsistent lables were normalized to single phases. NaN values have been preserved.

In [21]:
# Show various values of Funder types
df['Funder Type'].value_counts(dropna=False)


Funder Type
OTHER        375118
INDUSTRY     120900
OTHER_GOV     13654
NIH           10523
NETWORK        4638
FED            4572
INDIV           550
UNKNOWN          70
AMBIG             3
Name: count, dtype: int64

The only normalization to be performed for the Funder Type category is to group miscellaneous columns such as Unknown and Ambig into Other. NIH, FED, and OTHER_GOV are all government agencies, but are distinct enough to warrant their own categories, and combining them would erase structure. NETWORK represents research networks (distinct from governemnt) and will also be left as-is.

In [22]:
df['Funder Type'] = df['Funder Type'].replace({'INDIV': 'OTHER', 'UNKNOWN': 'OTHER', 'AMBIG': 'OTHER'})

In [23]:
# Show cleaned values of Funder types
df['Funder Type'].value_counts(dropna=False)

Funder Type
OTHER        375741
INDUSTRY     120900
OTHER_GOV     13654
NIH           10523
NETWORK        4638
FED            4572
Name: count, dtype: int64

## 9. Interventions Cleaning

Clinical trials frequently include multiple interventions, and analyzing at an intervention level (for example, answering which drugs and biologicals are studied across the most trials) becomes unreliable when storing multiple interventions in a single trial-level field in the `df` dataframe. To address this, interventions are separated on the vertical bar delimiter (|) and normalized into a separate intervention-levell, long format dataframe (`long_inter_df`) where each intervention is given its own row and linked back to the original clinical trial via the NCT Number. This transformation allows interventions to be parsed and re-classified reliably while preserving the one-to-many relationship between trials and interventions, rather than forcing multiple interventions into a single trial row.

After establishing the long format dataframe, the analysis aggregates back to the clinical trial level to create boolean indicator columns (has_drug, has_biological, has_drug_or_biological) that identify whether a clinical trial includes at least one drug or biological intervention. The column has_drug_or_biological and its boolean values are then used to then filter the original dataframe `df` down to only the clinical trials relevant to a study of drug development (those containing at least one DRUG or BIOLOGICAL intervention). All clinical trial rows not containing either a DRUG or BIOLOGICAL intervention are dropped, and the temporary columns created for this purpose (has_drug, has-biological, has_drug_or_biological) are subsequently removed.

Together, these steps produce two complementary dataframes: a cleaned trial-level dataframe (`df`) used for trial-based analysis and a normalized intervention-level dataframe (`long_inter_df`) used for intervention analysis (e.g., identifying which drugs or biologicals are studied most frequently). This process significantly reduces the size of the original df dataframe and also preserves the detailed intervention structure required for downstream analysis through the creation of long_inter_df. This process represents the most critical transformation step in the overall analysis.


In [24]:
# Current dataframe shape, for comparison at end of section
df.shape

(530028, 11)

In [32]:
# Reveal a small sample of current Intervention data formatting
df['Interventions'].sample(7)

391069    DIETARY_SUPPLEMENT: with Flavanol|DIETARY_SUPP...
79298     DRUG: IMC-001 and Optimal medical treatment|DR...
370299                                                  NaN
383029                  DRUG: Methylphenidate|DRUG: Placebo
245975            BEHAVIORAL: Access to sedentary behaviors
276916      DIETARY_SUPPLEMENT: L. paracasei fermented milk
513093                  PROCEDURE: Truwiev PCD laryngoscope
Name: Interventions, dtype: object

In [33]:
# Split the Interventions column based on the vertical bar delimiter ( | )
long_inter_df = df[['NCT Number', 'Interventions']].copy()

long_inter_df['Interventions'] = (
    long_inter_df['Interventions']
    .fillna('')
    .str.replace(r'\s*\|\s*', '|', regex=True)  # normalize spacing around the delimiter
    .str.split('|')
)

long_inter_df = long_inter_df.explode('Interventions')
long_inter_df['Interventions'] = long_inter_df['Interventions'].str.strip()
long_inter_df = long_inter_df[long_inter_df['Interventions'] != '']  # drop empty pieces


pattern = r'^\s*([A-Z_]+)\s*:\s*(.*)$'  # Regex, detailed explanation below:
# r'' defines the scope of what pandas is looking at (raw string)
# ^: Defines where regex should start matching from
# \s*: This allows for spaces, where \s defines a space and * defines potential multiples
# ([A-Z_]+) Begins capturing with an uppercase letter or underscore, and the + means one or more
# \s once again allows for a space after the intervention class but before the colon
# : matches the colon from the intervention column
# \s once again allows for a space, this time after the colon
# (.*) tells regex to capture everything it finds after the colon
# $ ends the string

long_inter_df[['Intervention_Class', 'Intervention_Name']] = long_inter_df['Interventions'].str.extract(pattern)
long_inter_df['Intervention_Name'] = long_inter_df['Intervention_Name'].str.strip()

# normalize class text (handles weird casing/spaces)
long_inter_df['Intervention_Class'] = long_inter_df['Intervention_Class'].str.strip().str.upper()




In [34]:
# Determine how many trial Interventions were unable to be extracted using regex in the above code block

failed_ext = long_inter_df['Intervention_Class'].isna()

print('Rows in new dataframe long_inter_df:', len(long_inter_df))
print('Rows with null value Interventions after extraction attempt:', failed_ext.sum())
print('Percent of Interventions that failed extraction:', round(100 * failed_ext.mean(), 2), '%')

# Value counts for unmatched value types
(long_inter_df.loc[failed_ext, 'Interventions'].value_counts().head(5))

Rows in new dataframe long_inter_df: 901606
Rows with null value Interventions after extraction attempt: 184
Percent of Interventions that failed extraction: 0.02 %


Interventions
OTHER              147
DEVICE               3
DIAGNOSTIC_TEST      3
PROCEDURE            3
GENETIC              2
Name: count, dtype: int64

The above output shows the regex extraction for Intervention (from dataframe `df` to dataframe `long_inter_df`) was successful at a rate of 99.98%. Remaining uncaptured Interventions are not clustered around DRUG or BIOLOGICAL, are inconsequentially small, and will therefore be dropped.

In [70]:
# Drop rows where Intervention extraction failed
long_inter_df = long_inter_df.dropna(subset=['Intervention_Class'])

In [72]:
# Verify the rows were dropped by comparing number of rows against the value output before the drop (901,606)
len(long_inter_df)

901422

In [35]:
# Identify trials with DRUG or BIOLOGICAL interventions
has_drug_or_bio = (long_inter_df['Intervention_Class'].isin(['DRUG', 'BIOLOGICAL'])
    .groupby(long_inter_df['NCT Number']).any()
    .rename('has_drug_or_biological').reset_index())

# Merge back onto the primary dataframe df
df = df.merge(has_drug_or_bio, on='NCT Number', how='left') # Left merge

# Assign the trials with no drug and no biological interventions a false boolean value
df['has_drug_or_biological'] = df['has_drug_or_biological'].fillna(False).astype(bool)

# Keep only trials with DRUG or BIOLOGICAL interventions
df = df[df['has_drug_or_biological']].copy()


C:\Users\jseat\AppData\Local\Temp\ipykernel_28180\778796617.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['has_drug_or_biological'] = df['has_drug_or_biological'].fillna(False).astype(bool)


While Intervention analysis can now be performed using the `long_inter_df` table, trial based analysis requires preserving a single row per clinical trial, as seen in the original dataframe `df`. The below code will aggregate drug and biological intervention names back to the trial level to support filtering out non-drug and non-biological trials. 

In [36]:
# Create a table that has all DRUG and BIOLOGICAL names for each trial
drug_bio_names = (
    long_inter_df.loc[long_inter_df['Intervention_Class'].isin(['DRUG', 'BIOLOGICAL'])]
    .groupby('NCT Number')['Intervention_Name']
    .apply(lambda s: sorted(set(s.dropna()))) # Condense the drug/bio name rows into one list per trial
    .reset_index(name='drug_bio_interventions')
)

# Left merge the intervention names back to the primary dataframe df using NCT Number as the key
df = df.merge(drug_bio_names, on='NCT Number', how='left')

In [ ]:
# Show sample values from newly created column
df['drug_bio_interventions'].sample(7)

196212    [CD0271 0.1%/CD1579 2.5% gel, CD0271 0.1%/CD15...
94625                                    [Allopregnanolone]
47536                   [0.5% 5-fluorouracil normal saline]
178946    [fluorouracil, gemcitabine hydrochloride, leuc...
20359                                  [Placebo, Prasugrel]
50004                                            [AndroGel]
155002                                 [LC51-0255, Placebo]
113577                               [Control, Remimazolam]
61834                                          [Radafaxine]
44504     [Azithromycin Capsule, Doxycycline Capsule, Pl...
151983    [cytarabine, decitabine, omacetaxine mepesucci...
Name: drug_bio_interventions, dtype: object

In [38]:
# Validate the newly created column 
df['has_drug_or_biological'].value_counts()

has_drug_or_biological
True    211771
Name: count, dtype: int64

In [39]:
# Show how many trials have various numbers of drug/biological interventions
df['drug_bio_interventions'].apply(len).value_counts().head(10)


drug_bio_interventions
1     96544
2     76398
3     21188
4      9401
5      3555
6      2019
7       991
8       613
9       317
10      223
Name: count, dtype: int64

In [40]:
# Small sample of the new dataframe
long_inter_df.sample(3)

,NCT Number,Interventions,Intervention_Class,Intervention_Name
375691,NCT02665572,BEHAVIORAL: bladder recycling,BEHAVIORAL,bladder recycling
338518,NCT04038008,DRUG: Diflucan®,DRUG,Diflucan®
315324,NCT06283355,BIOLOGICAL: Nasal Microbiota Transplant (NMT),BIOLOGICAL,Nasal Microbiota Transplant (NMT)


In [41]:
# Small sample of primary dataframe df with new columns
df.sample(3)

,NCT Number,Conditions,Interventions,Sponsor,Phases,Enrollment,Funder Type,Study Type,Start Date,Primary Completion Date,Locations,has_drug_or_biological,drug_bio_interventions
5177,NCT00003449,Ovarian Cancer,DRUG: dexamethasone|DRUG: gemcitabine hydrochl...,University of Southern California,PHASE2,35.0,OTHER,INTERVENTIONAL,1998-05-15,2002-06-15,USC/Norris Comprehensive Cancer Center and Hos...,True,"[dexamethasone, gemcitabine hydrochloride, pac..."
59031,NCT00711607,Healthy,DRUG: nomegestrol acetate and estradiol,Organon and Co,PHASE1,25.0,INDUSTRY,INTERVENTIONAL,2007-05-01,2007-09-01,NaN,True,[nomegestrol acetate and estradiol]
187013,NCT01638208,Irritable Bowel Syndrome,DRUG: VSL#3,"Asian Institute of Gastroenterology, India",PHASE4,40.0,OTHER,INTERVENTIONAL,2012-08-15,2012-12-15,"Asian Institute of Gstroenterology, Hyderabad,...",True,[VSL#3]


Of  all clinical trials included in the ClinicalTrials.gov dataset, approximately 211,000 (~45%) of them contain Drug or Biological Interventions. The remainder will now be dropped from the dataframe. 

In [42]:
df['has_drug_or_biological'] = (df['has_drug_or_biological'].fillna(False).astype('boolean')) # Define any trials with missing Intervention data a value of False in the 'has_drug_or_biological' column.
df = df[df['has_drug_or_biological']] # Drop all trials without Drug or Biological Interventions

In [44]:
df = df.drop(columns=['has_drug_or_biological']) # Drop columns that were created for this analysis

In [45]:
# Show new dataframe shape now that only drug and biological trials remain
df.shape

(211771, 12)

Value of 211,711 matches the output from cell 38. `df` successfully updated.

In [46]:
# Summary statistics on number of interventions per trial
long_inter_df.groupby('NCT Number').size().describe().round(2)

count    477836.00
mean          1.89
std           1.39
min           1.00
25%           1.00
50%           2.00
75%           2.00
max          97.00
dtype: float64

There now exists two dataframes for this clinical trial analysis. The original dataframe `df` has been reduced to 211,711 trials, all of which contain drug and/or biological interventions, and will be used for trial-level analysis. The newly created dataframe `long_inter_df` will be used to perform data analysis on the various biological and drug types studied across the clinical trials dataset. 

In [47]:
# Strip Interventions of whitespaces and transform to lowercase
long_inter_df['Interventions'] = (long_inter_df['Interventions'].str.strip().str.lower())

In [ ]:
# Investigate what the top interventions are across clinical trials
top_interventions = (long_inter_df['Interventions'].value_counts().head(10))

top_interventions

Interventions
drug: placebo                           27576
other: placebo                           6900
other: laboratory biomarker analysis     3105
dietary_supplement: placebo              2915
drug: cyclophosphamide                   2818
other: no intervention                   2251
drug: cisplatin                          2225
drug: carboplatin                        2211
drug: paclitaxel                         2082
drug: dexamethasone                      1997
Name: count, dtype: int64

The above output of the top ten interventions across all clinical trials reveals there are times where we see a single intervention repeated across mutliple classes, such as Placebo appearing as a DRUG, BIOLOGICAL, and DIETARY_SUPPLEMENT. Because of this, raw string-level counts at this stage mix together classification differences and should not be interpreted as accurate counts of intervention.

In the 04_analysis notebook, Interventions will be explicitly filtered by class using the long_inter_df table, where classification can be handled explicitly. This allows unique intervention types (e.g. Placebo) to be counted together when appearing under different classes in different trials. This approach avoids losing information during cleaning while allowing intervention counts to be interpreted correctly during analysis.

## 10. Conditions Cleaning

In [49]:
# Confirm missingness
df['Conditions'].isna().sum()

np.int64(1)

In [50]:
# Drop the single trial missing Condition data
df = df.dropna(subset=['Conditions'])

In [ ]:
# Show some sample Conditions
df['Conditions'].sample(5)

141717                            Breast Cancer
75357                       Alzheimer's Disease
52430                  KRAS Mutant Solid Tumors
122817                        Prostatic Adenoma
94242     Unrescetable Hepatocellular Carcinoma
Name: Conditions, dtype: object

The below code splits Condition on the vertical bar delimiter ( | ), explodes into one row per condition, cleans up formatting issues, and drops any created blank results.

In [52]:
long_cond_df = (df[['NCT Number', 'Conditions']].dropna(subset=['Conditions']).assign(Condition=lambda d: d['Conditions'].str.split(r"\|")).explode('Condition'))

long_cond_df['Condition'] = (long_cond_df['Condition'].astype(str).str.strip()
                             .str.replace(r"\s+", " ", regex=True) # normalize whitespace
                             .str.replace(r"\.$", "", regex=True)) # remove trailing period

# Drop blanks
long_cond_df = long_cond_df[long_cond_df['Condition'].ne("")]

In [53]:
# De-deuplicate in case the same condition appears twice in one trial
long_cond_df = long_cond_df.drop_duplicates(subset=['NCT Number', 'Condition'])

In [54]:
# Strip Condition of whitespaces and transform to lowercase
long_cond_df['Condition'] = (long_cond_df['Condition'].str.strip().str.lower())

# Investigate what the top conditions are across clinical trials
top_conditions = long_cond_df['Condition'].value_counts().head(25)

top_conditions

Condition
healthy                       6459
breast cancer                 3566
healthy volunteers            1938
prostate cancer               1893
pain                          1805
diabetes mellitus, type 2     1786
asthma                        1776
multiple myeloma              1713
hypertension                  1609
hiv infections                1583
schizophrenia                 1530
colorectal cancer             1522
lymphoma                      1499
covid-19                      1497
non-small cell lung cancer    1412
obesity                       1380
rheumatoid arthritis          1370
ovarian cancer                1273
lung cancer                   1230
leukemia                      1212
type 2 diabetes mellitus      1186
cancer                        1159
acute myeloid leukemia        1104
pancreatic cancer             1089
influenza                     1076
Name: count, dtype: int64

In the 04_analysis notebook, filtering Conditions will be done by matching portions of text to capture related conditions with differing phrasing. For example, filter on 'diabetes mellitus' to capture both 'type 2 diabetes mellitus' and 'diabetes mellitus, type 2' which are presently separate conditions.

## 11. Location Cleaning


In [55]:
# Confirm missingness
df['Locations'].isna().sum()


np.int64(16943)

In [56]:
df['Locations'].sample(7)

81382             Emergency Department, Nice, 06000, France
158399                                                  NaN
21963     Centre Hospitalier Universitaire de Québec, Qu...
77473     DIACARE, Ahemdabad, Gujarat, 380015, India|Guj...
23582     Creighton University Medical Center (including...
152165                                                  NaN
40105     Royal University Hospital, Saskatoon, Saskatch...
Name: Locations, dtype: object

No clinical trial rows will be removed due to missing Location data since those trials still contain important information for analyzing other categorical data. Some clinical trials are performed across multiple Locations, and the sample output above shows that the Country of the trial location is often the last word(s) within the data. 

The below code splits Location on the vertical bar delimiter ( | ), explodes into one row per condition, cleans up formatting issues, and drops any created blank results.

In [57]:
long_loc_df = (df[['NCT Number', 'Locations']].dropna(subset=['Locations']).assign(Location=lambda d: d['Locations'].str.split(r"\|")).explode('Location'))

long_loc_df['Location'] = (long_loc_df['Location'].astype(str).str.strip()
                             .str.replace(r"\s+", " ", regex=True) # normalize whitespace
                             .str.replace(r"\.$", "", regex=True)) # remove trailing period

# Drop blanks
long_loc_df = long_loc_df[long_loc_df['Location'].ne("")]

In [58]:
# Isolate Location on Country by extracting words after the last comma
long_loc_df['Country'] = (long_loc_df['Location'].str.split(',').str[-1].str.strip())

In [59]:
# De-deuplicate in case the same Country appears twice in one trial
long_loc_df = long_loc_df.drop_duplicates(subset=['NCT Number', 'Country'])

In [60]:
# Show a sample of extracted Country values
long_loc_df[['NCT Number', 'Country']].sample(10)

,NCT Number,Country
125945,NCT06529185,Egypt
130747,NCT01625182,Japan
167121,NCT05583006,Taiwan
173143,NCT03691831,United States
7417,NCT05562830,Denmark
137528,NCT00457691,Belgium
179394,NCT01164891,Switzerland
113275,NCT04139395,United States
90784,NCT01968954,South Korea
196191,NCT00501566,Romania


The Country of each clinical trial's location has been successfully extracted and input into a long format table long_loc_df, to be used for analysis in the 04_analysis notebook. Country names are converted to lowercase below for consistency in further analysis.

In [61]:
# Strip Country of whitespaces and transform to lowercase
long_loc_df['Country'] = (long_loc_df['Country'].str.strip().str.lower())

# Investigate what the top Countries are across clinical trials
top_countries = long_loc_df['Country'].value_counts().head(25)

top_countries

Country
united states     89553
china             25037
canada            15939
germany           15807
france            15794
united kingdom    14434
spain             13067
italy             11714
south korea       10895
australia          9074
belgium            8403
poland             8154
netherlands        7845
japan              7387
taiwan             5606
brazil             5402
russia             5386
denmark            5316
israel             5288
czechia            4990
hungary            4810
egypt              4694
sweden             4453
austria            4368
switzerland        4133
Name: count, dtype: int64

## 12. Enrollment Cleaning

In [62]:
# Confirm missingness
df['Enrollment'].isna().sum()

np.int64(2226)

In [63]:
# Show sample of enrollment values
df['Enrollment'].sample(5)

182267     30.0
109389    383.0
66452      10.0
142112     20.0
96614     450.0
Name: Enrollment, dtype: float64

Enrollment data is missing for approximately 1% of the trials (2,226 trials missing data, of remaining 211,711 trials). No trials will be removed due to missing Enrollment data since those trials still contain important information for analyzing other categorical data. The enrollment values currently exist as float datatype, but will be converted to integers since all values for trial enrollments should be whole numbers.

In [64]:
# Cast enrollment values to integers (whole numbers)
df['Enrollment'] = df['Enrollment'].astype('Int64')

In [65]:
# Investigate what the top Enrollment counts are across clinical trials
top_enrollments = df['Enrollment'].value_counts().head(25)

top_enrollments

Enrollment
0      7593
30     6226
60     5774
20     5601
40     5509
50     4094
100    4079
24     3920
12     3237
10     3222
80     2737
120    2476
18     2415
15     2406
36     2360
16     2264
200    2238
32     2121
25     2090
48     1997
6      1875
90     1820
150    1688
28     1615
8      1613
Name: count, dtype: Int64

Enrollment values of zero indicate trials that have been withdrawn or terminated. The Enrollment category is now ready for use in the  04_analysis notebook.

## 13. Final Checks and Summary

This notebook performed the main cleaning and transformations required to enable a successful analysis of the ClinicalTrials.gov dataset, in the context of investigating global drug development patterns. The original dataframe `df` contained multiple forms of messy or inconsistent data, including missing values, mixed data types, and columns that had one-to-many relationships (Interventions, Conditions, and Locations).

Key cleaning steps included standardizing date fields, validating enrollment values, normalizing categorical variables, removing irrelevant columns, and handling columns with multiple values that required further parsing for detailed analysis. The most significant transformations involved normalizing the Interventions, Conditions, and Locations columns into three separate long format tables, details of which are shown below in this section. The resulting processed dataset `df` preserves the original data structure while establishing consistent types, formats, and assumptions, providing a basis for analysis in later notebooks.

## Outputs & Handoff to 04_analysis

This notebook produces four cleaned datasets used as inputs to `04_analysis.ipynb`:

- **Trial-level dataset (`df`)** is the primary dataframe for analysis, and is based on one row per clinical trial with unique `NCT Number`.
- **Interventions (`long_inter_df`)** normalized the `Interventions` field into a long format, with one row per intervention per trial.
- **Conditions (`long_cond_df`)** normalized the `Conditions` field into a long format, with one row per condition per trial.
- **Locations (`long_loc_df`)** normalized the `Locations` field into a long format, with one row per location per trial, with country extracted.

Data Dictionaries for each of the four cleaned datasets are shown below.

## Final Cleaned Trial-Level Dataset (`df`) — Data Dictionary

In [66]:
print('Dateframe: df')
print('rows:', df.shape[0], 'cols:', df.shape[1])

Dateframe: df
rows: 211770 cols: 12


| Column | Description |
|---|---|
| `NCT Number` | Unique ClinicalTrials.gov identifier for each trial (primary key). |
| `Conditions` | Pipe-delimited list of conditions/diseases studied in the trial (raw text, preserved). |
| `Interventions` | Pipe-delimited list of intervention strings as reported in ClinicalTrials.gov (raw text, preserved). |
| `Sponsor` | Lead sponsor organization responsible for the trial. |
| `Phases` | Clinical trial phase(s) (cleaned and standardized text). |
| `Enrollment` | Planned or actual number of participants enrolled (numeric, non-negative). |
| `Funder Type` | Categorized funding source (e.g., Industry, NIH, Other). |
| `Study Type` | Type of study (e.g., Interventional, Observational). |
| `Start Date` | Trial start date (parsed to datetime where possible). |
| `Primary Completion Date` | Date of final data collection for the primary outcome measure. |
| `Locations` | Pipe-delimited list of study site locations as reported. |
| `drug_bio_interventions` | List of unique DRUG and/or BIOLOGICAL intervention names associated with the trial (derived from normalized intervention data). |

---

## Normalized Intervention-Level Dataset (`long_inter_df`) — Data Dictionary

In [68]:
print('Dateframe: long_inter_df')
print('rows:', long_inter_df.shape[0], 'cols:', long_inter_df.shape[1])
long_inter_df.columns.tolist()

Dateframe: long_inter_df
rows: 901606 cols: 4


['NCT Number', 'Interventions', 'Intervention_Class', 'Intervention_Name']

| Column | Description |
|---|---|
| `NCT Number` | Trial identifier (foreign key linking back to the trial-level `df`). |
| `Interventions` | Original interventions string from the trial-level dataset `df`. |
| `Intervention_Class` | Intervention type (e.g., `DRUG`, `BIOLOGICAL`) using regex. |
| `Intervention_Name` | Intervention name extracted from each intervention entry (text after the `:`) and cleaned (trimmed/normalized). |

---

## Normalized Condition-Level Dataset (`long_cond_df`) — Data Dictionary

In [69]:
print('Dataframe: long_cond_df')
print('rows:', long_cond_df.shape[0], 'cols:', long_cond_df.shape[1])
long_cond_df.columns.tolist()

Dataframe: long_cond_df
rows: 362657 cols: 3


['NCT Number', 'Conditions', 'Condition']

| Column | Description |
|---|---|
| `NCT Number` | Trial identifier (foreign key linking back to the trial-level `df`). |
| `Conditions` | Original conditions string from the trial-level dataset `df`. |
| `Condition` | Individual condition value derived by splitting `Conditions`. |

---

## Normalized Location-Level Dataset (`long_loc_df`) — Data Dictionary

In [70]:
print('Dataframe: long_loc_df')
print('rows:', long_loc_df.shape[0], 'cols:', long_loc_df.shape[1])
long_loc_df.columns.tolist()

Dataframe: long_loc_df
rows: 385273 cols: 4


['NCT Number', 'Locations', 'Location', 'Country']

| Column | Description |
|---|---|
| `NCT Number` | Trial identifier (foreign key linking back to the trial-level `df`). |
| `Locations` | Original locations string from the trial-level dataset `df`. |
| `Location` | Single location value derived by splitting `Locations`. |
| `Country` | Country extracted from `Location`. |

## 14. Export to CSV

The three new dataframes and the processed primary dataframe will now be exported to the project's processed data folder for use in the 04_analysis notebook.

In [74]:
from pathlib import Path

project_root = Path.cwd().parent
output_dir = project_root/'data'/'processed'
output_dir.mkdir(parents=True, exist_ok=True)

df.to_csv(output_dir/'clean_trials.csv', index=False)
long_inter_df.to_csv(output_dir/'clean_interventions.csv', index=False)
long_cond_df.to_csv(output_dir/'clean_conditions.csv', index=False)
long_loc_df.to_csv(output_dir/'clean_locations.csv', index=False)